In [11]:
import pdb
import pycm
import sys
sys.path.append('../')
from glob import glob
import torch
import scipy
from torch import nn
from torch.nn import *
from torch.nn import functional as F
import cv2
import os
# from models import Model
from PIL import Image
from skimage import io, transform
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from torchvision.datasets.folder import pil_loader
from torchvision import transforms
import random
import numpy as np
from torchsummary import summary
import pretrainedmodels
from sklearn.model_selection import KFold, StratifiedKFold
from efficientnet_pytorch import EfficientNet
import albumentations
from pycm import ConfusionMatrix
from tensorboard_logger import * 
import warnings
warnings.filterwarnings("ignore")
from models import *
import albumentations
from albumentations import torch as AT
os.environ['CUDA_VISIBLE_DEVICES']=""
from dataloader import provider
from mask_functions import *
import warnings
# warnings.filterwarnings('ignore', category=RuntimeWarning)

%matplotlib inline


In [12]:

LOGIT_TO_IMAGE_SCALE = 4


##############################################################

class Conv2dBn(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size=3, padding=1, stride=1):
        super(Conv2dBn, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size,
                              padding=padding, stride=stride, bias=False)
        self.bn = BatchNorm2d(out_channel, eps=1e-05, momentum=0.1)

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        return x


class Basic(nn.Module):
    def __init__(self, in_channel, out_channel, stride=1, is_shortcut=False):
        super(Basic, self).__init__()
        self.is_shortcut = in_channel != out_channel or stride != 1
        if self.is_shortcut:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1,
                          padding=0, stride=stride, bias=False),
                BatchNorm2d(out_channel)
            )

        self.conv1 = nn.Conv2d(
            in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(
            out_channel, out_channel, kernel_size=3, stride=1,      padding=1, bias=False)
        self.bn2 = BatchNorm2d(out_channel)

    def forward(self, x):

        if self.is_shortcut:
            shortcut = self.shortcut(x)
        else:
            shortcut = x

        x = self.bn1(self.conv1(x))
        x = F.relu(x, inplace=True)
        x = self.bn2(self.conv2(x)) + shortcut
        x = F.relu(x, inplace=True)
        return x


class Decode(nn.Module):
    def __init__(self, in_channel, channel, out_channel):
        super(Decode, self).__init__()
        self.top = nn.Sequential(
            nn.Conv2d(in_channel+channel, out_channel, kernel_size=3,
                      stride=1, padding=1, bias=False),
            BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channel, out_channel, kernel_size=1,
                      stride=1, padding=0, bias=False),
            BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )

    def forward(self, x, e):
        batch_size, C, H, W = e.shape
        x = F.interpolate(x, size=(H, W), mode='bilinear', align_corners=False)
        x = torch.cat([x, e], 1)
        x = self.top(x)
        return x


##############################################################


class UNet(nn.Module):

    def __init__(self, in_channel=1, num_class=1):
        super(UNet, self).__init__()

        self.encode = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channel, 64, kernel_size=7,
                          stride=2, padding=3, bias=False),
                BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2, stride=2,)
            )
        ])
        for in_channel, out_channel, stride, num_block in [
            [64,          64,     1,       3],
            [64,         128,     2,       4],
            [128,         256,     2,       6],
            [256,         512,     2,       3],
            [512,         512,     2,       2],
            [512,         512,     2,       2],
        ]:
            self.encode.append(
                nn.Sequential(
                    Basic(in_channel, out_channel,  stride=stride, ),
                    *[Basic(out_channel, out_channel,  stride=1,) for i in range(1, num_block)]
                )
            )

        self.decode = nn.ModuleList([
            Decode(512, 512, 256),
            Decode(256, 512, 256),
            Decode(256, 256, 128),
            Decode(128, 128, 64),
            Decode(64, 64, 32),
        ])

        self.logit = nn.Conv2d(
            32, num_class, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        batch_size, C, H, W = x.shape

        x = self.encode[0](x)  # print('encode[0] :', x.shape)
        x = self.encode[1](x)
        e1 = x  # ; print('encode[1] :', x.shape)
        x = self.encode[2](x)
        e2 = x  # ; print('encode[2] :', x.shape)
        x = self.encode[3](x)
        e3 = x  # ; print('encode[3] :', x.shape)
        x = self.encode[4](x)
        e4 = x  # ; print('encode[4] :', x.shape)
        x = self.encode[5](x)
        e5 = x  # ; print('encode[5] :', x.shape)
        x = self.encode[6](x)  # print('encode[6] :', x.shape)

        x = self.decode[0](x, e5)  # ; print('decode[0] :', x.shape)
        x = self.decode[1](x, e4)  # ; print('decode[1] :', x.shape)
        x = self.decode[2](x, e3)  # ; print('decode[2] :', x.shape)
        x = self.decode[3](x, e2)  # ; print('decode[3] :', x.shape)
        x = self.decode[4](x, e1)  # ; print('decode[4] :', x.shape)

        #x = F.dropout(x, 0.5, training=self.training)
        logit = self.logit(x)
        return logit



In [18]:
model = UNet(1, 1)

In [14]:
model

UNet(
  (encode): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Basic(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Basic(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): C

In [21]:
summary(model, input_size=(1, 1024, 1024))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           3,136
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
         MaxPool2d-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 256, 256]          36,864
       BatchNorm2d-6         [-1, 64, 256, 256]             128
            Conv2d-7         [-1, 64, 256, 256]          36,864
       BatchNorm2d-8         [-1, 64, 256, 256]             128
             Basic-9         [-1, 64, 256, 256]               0
           Conv2d-10         [-1, 64, 256, 256]          36,864
      BatchNorm2d-11         [-1, 64, 256, 256]             128
           Conv2d-12         [-1, 64, 256, 256]          36,864
      BatchNorm2d-13         [-1, 64, 256, 256]             128
            Basic-14         [-1, 64, 2

In [20]:
from models import get_model
model1 = get_model("UNet", 1)
summary(model1, input_size=(1, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
         MaxPool2d-5         [-1, 64, 128, 128]               0
            Conv2d-6        [-1, 128, 128, 128]          73,856
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]         147,584
              ReLU-9        [-1, 128, 128, 128]               0
        MaxPool2d-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         295,168
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 256, 64, 64]         590,080
             ReLU-14          [-1, 256,

In [27]:
model1 = UNet2(1, 1) 
summary(model1, input_size=(1, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
       double_conv-7         [-1, 64, 256, 256]               0
            inconv-8         [-1, 64, 256, 256]               0
         MaxPool2d-9         [-1, 64, 128, 128]               0
           Conv2d-10        [-1, 128, 128, 128]          73,856
      BatchNorm2d-11        [-1, 128, 128, 128]             256
             ReLU-12        [-1, 128, 128, 128]               0
           Conv2d-13        [-1, 128, 128, 128]         147,584
      BatchNorm2d-14        [-1, 128, 1

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x
    
class UNet2(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet2, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return F.sigmoid(x)
# https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_parts.py

In [22]:
from torchvision import models

In [23]:
dlab = models.segmentation.deeplabv3_resnet101(pretrained=1).eval()

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /home/ags/.cache/torch/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 244545539/244545539 [00:11<00:00, 21835768.70it/s]
